importacion de librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./data/2024_Accidentalidad.csv', delimiter=';')


In [61]:
df.sample(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
21146,2024S018849,04/06/2024,17:25:00,"CALL. BEATRIZ DE BOBADILLA, 15",15,9,MONCLOA-ARAVACA,Alcance,Despejado,Turismos y vehículos ligeros,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,439275.0,4477793.0,N,NaN
826,2024S000532,08/01/2024,17:30:00,AVDA. EURO / AVDA. CARABANCHEL ALTO,70,11,CARABANCHEL,Alcance,NaN,Vehículos de carga,Conductor,De 45 a 49 años,Hombre,14.0,Sin asistencia sanitaria,436021.0,4468420.0,N,NaN
11563,2024S010965,27/03/2024,12:15:00,"AVDA. ARAGON, 359",359,21,BARAJAS,Colisión lateral,NaN,Vehículos de carga,Pasajero,De 40 a 44 años,Hombre,NaN,NaN,454754.0,4477501.0,N,NaN
26734,2024S022129,14/07/2024,23:45:00,AVDA. CAMINO DE SANTIAGO / CALL. OBANOS,13,8,FUENCARRAL-EL PARDO,Colisión fronto-lateral,Despejado,Motocicletas y ciclomotores,Conductor,De 35 a 39 años,Hombre,3.0,Ingreso superior a 24 horas,442854.0,4483266.0,N,NaN
8604,2024S008209,02/03/2024,14:05:00,PASEO. MORET / CALL. PRINCESA,1,9,MONCLOA-ARAVACA,Otro,Despejado,Turismos y vehículos ligeros,Conductor,De 60 a 64 años,Hombre,NaN,NaN,438966.0,4476272.0,N,NaN
24081,2024S020611,24/06/2024,17:00:00,"CALL. PRINCESA, 2",2,1,CENTRO,Colisión fronto-lateral,Despejado,Turismos y vehículos ligeros,Conductor,De 60 a 64 años,Hombre,14.0,Sin asistencia sanitaria,439613.0,4475128.0,N,NaN
29839,2024S024066,12/08/2024,21:24:00,"PASEO. SAN FRANCISCO DE SALES, 23A",23A,7,CHAMBERÍ,Atropello a persona,Despejado,Turismos y vehículos ligeros,Peatón,De 60 a 64 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,439404.0,4476998.0,N,NaN
853,2024S000549,08/01/2024,19:55:00,"CALL. SOCUELLAMOS, 7",7,12,USERA,Alcance,Despejado,Turismos y vehículos ligeros,Conductor,De 30 a 34 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,440888.0,4470255.0,N,NaN
24048,2024S020598,24/06/2024,18:30:00,"AUTOV. A-42, +00500E",+00500E,12,USERA,Colisión múltiple,Despejado,Turismos y vehículos ligeros,Pasajero,Más de 74 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,438876.0,4469159.0,N,NaN
27068,2024S022325,17/07/2024,18:10:00,"CALL. DOCTOR ZAMENHOF, 22",22,20,SAN BLAS-CANILLEJAS,Caída,Despejado,Bicicletas y ciclos no motorizados,Conductor,De 25 a 29 años,Hombre,5.0,Asistencia sanitaria ambulatoria con posterior...,446837.0,4477549.0,N,NaN


Analisis del dataset

https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_ConjuntoDatos_Accidentesv2.pdf

Lo primero que nos damos cuenta cuando leemos el anterior documento es que el año del accidente no viene en la fecha, si no que viene en los 4 primeros char del numero del expediente.

**localizacion** y **numero**: con la localizacion y el numero (numero de la calle) nos parece interesante quedarnosla, pero solo para la parte de entendimiento de los datos del cluster, ya que para el claster es mas util las coordenadas utm

para el codigo del distrito nos va a servir para hacer el cluster, pero como el numero en si no nos dice nada, pues entonces lo que vamos a hacer un mapeo del distrito para entender y tener la correlacion

para el tipo de accidente y para el estado metereologico haremos un mapeo simple de las 13 posibles variables (mapping)

en tipo de vehiculo habia demasiados valroes unicos (33) por lo que hemos intentado generalizar un poco para poder tratar con esta columna, haciendo un mapeo y un posible posterior onehotencoding

para el **rango de edad** vamos 

Entendimiento del Dataset

Limpieza

Explicacion de las acciones realizadas


In [54]:
len(df['tipo_vehiculo'].unique())


33

In [56]:
df['tipo_vehiculo'].value_counts()

tipo_vehiculo
Turismo                              27406
Motocicleta hasta 125cc               2668
Furgoneta                             2667
Motocicleta > 125cc                   1469
Autobús                               1264
Camión rígido                          973
Todo terreno                           706
Bicicleta                              670
VMU eléctrico                          665
Ciclomotor                             387
Bicicleta EPAC (pedaleo asistido)      211
Tractocamión                           187
Maquinaria de obras                    151
Otros vehículos con motor              131
Vehículo articulado                     70
Autobús articulado                      69
Sin especificar                         40
Cuadriciclo ligero                      32
Autobus EMT                             17
Ciclo                                   13
Cuadriciclo no ligero                   12
Autocaravana                            11
Otros vehículos sin motor               

In [57]:
map_vehiculos = {
    'Turismo': 'Turismos y vehículos ligeros',
    'Todo terreno': 'Turismos y vehículos ligeros',
    'Autocaravana': 'Turismos y vehículos ligeros',
    'Motocicleta hasta 125cc': 'Motocicletas y ciclomotores',
    'Motocicleta > 125cc': 'Motocicletas y ciclomotores',
    'Ciclomotor': 'Motocicletas y ciclomotores',
    'Bicicleta EPAC (pedaleo asistido)': 'Motocicletas y ciclomotores',
    'Ciclomotor de tres ruedas': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas hasta 125cc': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas > 125cc': 'Motocicletas y ciclomotores',
    'Ciclo de motor L1e-A': 'Motocicletas y ciclomotores',
    'Ciclomotor de dos ruedas L1e-B': 'Motocicletas y ciclomotores',
    'Furgoneta': 'Vehículos de carga',
    'Camión rígido': 'Vehículos de carga',
    'Tractocamión': 'Vehículos de carga',
    'Vehículo articulado': 'Vehículos de carga',
    'Semiremolque': 'Vehículos de carga',
    'Remolque': 'Vehículos de carga',
    'Autobús': 'Vehículos de transporte público',
    'Autobús articulado': 'Vehículos de transporte público',
    'Autobus EMT': 'Vehículos de transporte público',
    'Bicicleta': 'Bicicletas y ciclos no motorizados',
    'Ciclo': 'Bicicletas y ciclos no motorizados',
    'Patinete no eléctrico': 'Bicicletas y ciclos no motorizados',
    'VMU eléctrico': 'Vehículos eléctricos ligeros',
    'Maquinaria de obras': 'Vehículos especializados',
    'Ambulancia SAMUR': 'Vehículos especializados',
    'Otros vehículos con motor': 'Vehículos especializados',
    'Otros vehículos sin motor': 'Vehículos especializados',
    'Sin especificar': 'Sin clasificar/especificar',
    'Cuadriciclo ligero': 'Sin clasificar/especificar',
    'Cuadriciclo no ligero': 'Sin clasificar/especificar'
}

In [ ]:
df['tipo_vehiculo'] = df['tipo_vehiculo'].map(map_vehiculos)

In [63]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,De 55 a 59 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,444913.0,4481427.0,N,NaN
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,Conductor,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,444913.0,4481427.0,N,NaN
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,Conductor,De 25 a 29 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,440123.0,4475170.0,N,NaN
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,De 35 a 39 años,Hombre,14.0,Sin asistencia sanitaria,440123.0,4475170.0,N,NaN
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,De 25 a 29 años,Hombre,NaN,NaN,440137.0,4475721.0,N,NaN
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,De 55 a 59 años,Hombre,NaN,NaN,440137.0,4475721.0,N,NaN
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos de carga,Conductor,De 25 a 29 años,Hombre,14.0,Sin asistencia sanitaria,440709.0,4480102.0,N,NaN
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos especializados,Conductor,De 50 a 54 años,Hombre,14.0,Sin asistencia sanitaria,440709.0,4480102.0,N,NaN
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Vehículos de carga,Conductor,De 60 a 64 años,Hombre,NaN,NaN,433983.0,4468473.0,N,NaN
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Turismos y vehículos ligeros,Conductor,De 25 a 29 años,Hombre,NaN,NaN,433983.0,4468473.0,N,NaN


In [19]:
df['rango_edad'].value_counts()

rango_edad
Desconocido        4316
De 45 a 49 años    4107
De 30 a 34 años    3919
De 25 a 29 años    3885
De 40 a 44 años    3845
De 35 a 39 años    3796
De 50 a 54 años    3698
De 55 a 59 años    2869
De 21 a 24 años    2627
De 60 a 64 años    1990
De 18 a 20 años    1268
Más de 74 años      963
De 65 a 69 años     956
De 70 a 74 años     576
De 15 a 17 años     374
Menor de 5 años     367
De 10 a 14 años     349
De 6 a 9 años       260
Name: count, dtype: int64

In [20]:
df.columns

Index(['fecha', 'hora', 'cod_distrito', 'tipo_accidente',
       'estado_meteorológico', 'tipo_vehiculo', 'tipo_persona', 'rango_edad',
       'sexo', 'cod_lesividad', 'positiva_alcohol', 'positiva_droga'],
      dtype='object')

In [66]:
df['cod_lesividad'].unique()

array([ 2., 14.,  7., nan,  1.,  3.,  6.,  5.,  4.])